In [59]:
# Day 15 part 1
import numpy as np
import networkx as nx

def read_input_data(filename):
    with open(filename, 'r') as in_file:
        raw_data = in_file.read().split('\n')
    data = [list(row) for row in raw_data]
    return np.array(data, dtype=np.int32)


def get_lowest_risk_path(data):
    graph = nx.grid_2d_graph(*data.shape, create_using=nx.DiGraph)
    for (_, v), e in graph.edges.items():
        e["weight"] = data[v]
    source, *_, target = graph.nodes
    return nx.shortest_path_length(graph, source, target, weight="weight")


# Test first
test_data = read_input_data('test_data/day15.txt')
print("Running on test data. Verifying result...")
print(get_lowest_risk_path(test_data) == 40)
print('')

# Now run on actual data
real_data = read_input_data('data/day15.txt')
result = get_lowest_risk_path(real_data)
print("Puzzle answer is: " + str(result))

Running on test data. Verifying result...
True

Puzzle answer is: 621


In [58]:
def expand_map(data):
    data = np.vstack([np.hstack([data + i + j for i in range(5)]) for j in range(5)])
    data[data > 9] -= 9
    return data

# Test first
test_data = read_input_data('test_data/day15.txt')
test_data = expand_map(test_data)
print(get_lowest_risk_path(test_data) == 315)
print('')

# Now run on actual data
real_data = read_input_data('data/day15.txt')
real_data = expand_map(real_data)
result = get_lowest_risk_path(real_data)
print("Puzzle answer is: " + str(result))

True

Puzzle answer is: 2904


In [206]:
# Day 16 Part 1

def read_input_data(filename):
    with open(filename, 'r') as in_file:
        data = in_file.read()
    return data

def hex_to_binary(hex_packet):
    return bin(int(hex_packet, 16))[2:].zfill(len(hex_packet) * 4)

def consume_and_read(bin_packet, n_bits):
    return bin_packet[n_bits:], int(bin_packet[:n_bits], 2)

def consume_and_get_next_literal_number(bin_packet):
    return bin_packet[5:], int(bin_packet[1:5], 2), True if int(bin_packet[0], 2) == 1 else False

def consume_and_process_literal_packet(bin_packet):
    literal_values_left = True
    n_literals = 0
    while(literal_values_left):
        n_literals = n_literals + 1
        bin_packet, curr_literal, literal_values_left = consume_and_get_next_literal_number(bin_packet)
    return bin_packet
    

def recursive_process_packets(binary_packet, version_total):
    
    while(len(binary_packet) > 0):
        
        if int(binary_packet, 2) == 0:
            break
        
        binary_packet, curr_version = consume_and_read(binary_packet, 3)
        binary_packet, curr_type = consume_and_read(binary_packet, 3)
        version_total = version_total + curr_version
        
        if curr_type == 4:
            #print("Literal packet. Version " + str(curr_version))
            binary_packet = consume_and_process_literal_packet(binary_packet)
            #print("Adding version: " + str(curr_version))
            #version_total = version_total + curr_version
        else:
            binary_packet, length_type_ID = consume_and_read(binary_packet, 1)
            
            if length_type_ID == 0:
                binary_packet, subpacket_length = consume_and_read(binary_packet, 15)
                #print("Operator packet with subpackets of length " + str(subpacket_length) + ". Version " + str(curr_version))
                subpackets = binary_packet[:subpacket_length]
                binary_packet = binary_packet[subpacket_length:]
                #print("recursing: 15 bits ahead")
                
                _, version_total = recursive_process_packets(subpackets, version_total)
                #version_total = curr_version + recur_version
                #print("Adding version: " + str(curr_version))
                #print("New total: " + str(version_total))
            
            elif length_type_ID == 1:
                binary_packet, n_subpackets = consume_and_read(binary_packet, 11)
                #print("Operator packet with " + str(n_subpackets) + " subpackets. Version " + str(curr_version))
                for i in range(0, n_subpackets):
                    binary_packet, version_total = recursive_process_packets(binary_packet, version_total)
                    #version_total = curr_version + recur_version
                    #print("Adding version: " + str(curr_version))
                    #print("New total: " + str(version_total))

    return binary_packet, version_total
        
    
def add_version_numbers(hex_big_packet):
    binary_big_packet = hex_to_binary(hex_big_packet)
    #print(binary_big_packet)
    _, version_total = recursive_process_packets(binary_big_packet, 0)
    #print(version_total)
    return version_total
    
        
# Test first
print(add_version_numbers(read_input_data('test_data/day16a.txt')) == 16)
print(add_version_numbers(read_input_data('test_data/day16b.txt')) == 12)
print(add_version_numbers(read_input_data('test_data/day16c.txt')) == 23)
print(add_version_numbers(read_input_data('test_data/day16d.txt')) == 31)
print('')

real_data = read_input_data('data/day16.txt')
result = add_version_numbers(real_data)
print("Puzzle answer is: " + str(result))

True
True
True
True

Puzzle answer is: 847


In [213]:
# Day 16 Part 2
import numpy as np

def consume_and_get_next_literal_number(bin_packet):
    return bin_packet[5:], bin_packet[1:5], True if int(bin_packet[0], 2) == 1 else False

def consume_and_process_literal_packet(bin_packet):
    whole_literal = 0
    literal_values_left = True
    n_literals = 0
    while(literal_values_left):
        n_literals = n_literals + 1
        bin_packet, curr_literal, literal_values_left = consume_and_get_next_literal_number(bin_packet)
        whole_literal = str(whole_literal) + str(curr_literal)
    return bin_packet, int(whole_literal, 2)
       
def single_operation(op, arguments):
    #print(op, arguments)
    if op == 0:
        return sum(arguments)
    elif op == 1:
        return np.prod(arguments)
    elif op == 2:
        return min(arguments)
    elif op == 3:
        return max(arguments)
    elif op == 5:
        return 1 if arguments[0] > arguments[1] else 0
    elif op == 6:
        return 1 if arguments[0] < arguments[1] else 0
    elif op == 7:
        return 1 if arguments[0] == arguments[1] else 0
    
def recursive_perform_operation(binary_packet, args_list, max_packets = 1e5):
    
    n_packets_found = 0
    
    while(len(binary_packet) > 0 and n_packets_found < max_packets):
        if int(binary_packet, 2) == 0:
            break
        
        binary_packet, curr_version = consume_and_read(binary_packet, 3)
        binary_packet, curr_type = consume_and_read(binary_packet, 3)
        n_packets_found = n_packets_found + 1
        
        if curr_type == 4:
            binary_packet, current_total = consume_and_process_literal_packet(binary_packet)
            args_list.append(current_total)
            #print("Literal packet. Literal " + str(current_total))
        else:
            binary_packet, length_type_ID = consume_and_read(binary_packet, 1)
            
            if length_type_ID == 0:
                binary_packet, subpacket_length = consume_and_read(binary_packet, 15)
                #print("Operator packet with subpackets of length " + str(subpacket_length) + ". Type " + str(curr_type))
                subpackets = binary_packet[:subpacket_length]
                binary_packet = binary_packet[subpacket_length:]
                args = []
                _, args = recursive_perform_operation(subpackets, args)
                result = single_operation(curr_type, args)
                args_list.append(result)
            
            elif length_type_ID == 1:
                binary_packet, n_subpackets = consume_and_read(binary_packet, 11)
                #print("Operator packet with " + str(n_subpackets) + " subpackets. Type " + str(curr_type))
                args = []
                for i in range(0, n_subpackets):
                    binary_packet, args = recursive_perform_operation(binary_packet, args, 1)
                result = single_operation(curr_type, args)
                args_list.append(result)

    return binary_packet, args_list

def perform_operation(hex_big_packet):
    binary_big_packet = hex_to_binary(hex_big_packet)
    _, result = recursive_perform_operation(binary_big_packet, [])
    return result[0]

# Test first
print(perform_operation("C200B40A82") == 3)
print(perform_operation("04005AC33890") == 54)
print(perform_operation("880086C3E88112") == 7)
print(perform_operation("CE00C43D881120") == 9)
print(perform_operation("D8005AC2A8F0") == 1)
print(perform_operation("F600BC2D8F") == 0)
print(perform_operation("9C005AC2F8F0") == 0)
print(perform_operation("9C0141080250320F1802104A08") == 1)
print('')

real_data = read_input_data('data/day16.txt')
result = perform_operation(real_data)
print("Puzzle answer is: " + str(result))

True
True
True
True
True
True
True
True

Puzzle answer is: 333794664059


In [250]:
# Day 17 Part 1

from math import sqrt


def step(position, velocity):
    position[0] = position[0] + velocity[0]
    position[1] = position[1] + velocity[1]
    if velocity[0] != 0:
        velocity[0] = abs(velocity[0]) - 1 * (1 - (velocity[0] <= 0))
    velocity[1] = velocity[1] - 1

def its_a_hit(position, target_area):
    return position[0] >= target_area[0] and position[0] <= target_area[1] and position[1] >= target_area[2] and position[1] <= target_area[3]
    
def its_a_miss(position, target_area):
    return position[0] > target_area[1] or position[1] < target_area[2]

def evaluate_path(initial_velocity, target_area):
    position = [0, 0]
    velocity = initial_velocity.copy()
    
    while True:
        step(position, velocity)
        if its_a_hit(position, target_area):
            return True, initial_velocity[1]*(initial_velocity[1] + 1) / 2
        elif its_a_miss(position, target_area):
            return False, 0
        

def sling_with_style(target_area):
    # No point having negative y velocities.
    # For now we will assume that we want the probe to go flying as high as possible then do a dead drop into the target area
    # This means that the residual x velocity is zero, so we have a range of possible x velocities. Are they even relevant?
    min_x_velocity = int(sqrt(target_area[0] * 2))
    max_x_velocity = int(sqrt(target_area[1] * 2) + 1)
    
    min_y_velocity = 0
    max_y_velocity = abs(target_area[2]) + 1
    max_height = 0
    
    for x in range(min_x_velocity, max_x_velocity + 1):
        for y in range(min_y_velocity, max_y_velocity):
            valid, height = evaluate_path([x, y], target_area)
            if valid:
                max_height = max(max_height, height)
                
    return max_height


# Test first
print(sling_with_style((20, 30, -10, -5)) == 45)

result = sling_with_style((209, 238, -86, -59))
print("Puzzle answer is: " + str(result))

True
Puzzle answer is: 3655.0


In [253]:
# Day 17 Part 2

# Warning for brute force....

def count_all_hits(target_area):
    # The assumption we made in part 1 is no longer valid, we need to widen our search
    min_x_velocity = int(sqrt(target_area[0] * 2))
    max_x_velocity = target_area[1] + 1
    
    min_y_velocity = target_area[2]
    max_y_velocity = abs(target_area[2]) + 1
    
    valid_count = 0
    
    for x in range(min_x_velocity, max_x_velocity + 1):
        for y in range(min_y_velocity, max_y_velocity):
            valid, height = evaluate_path([x, y], target_area)
            if valid:
                valid_count = valid_count + 1
    print(valid_count)
    return valid_count
    
# Test first
print(count_all_hits((20, 30, -10, -5)) == 112)

result = count_all_hits((209, 238, -86, -59))
print("Puzzle answer is: " + str(result))

112
True
1447
Puzzle answer is: 1447
